# GOAL


# MODEL DESCRIPTION


# PACKAGES


In [1]:
import salabim as sim
import math
from scipy.stats import norm

# FIXED PARAMETERS


In [2]:
env = sim.Environment(trace=True)

# define replenishment parameters
param_lead_time = 5
param_order_frequency = 3
param_order_rounding = 5
param_safety_service_level = 0.95

planning_horizon = param_order_frequency + param_lead_time

line#        time current component    action                               information
------ ---------- -------------------- -----------------------------------  ------------------------------------------------
                                       line numbers refers to               1013671131.py
    1                                  default environment initialize       
    1                                  main create                          
    1       0.000 main                 current                              


# DISTRIBUTIONS


In [3]:
# define functions
function_demand = sim.Bounded(
    sim.Normal(mean=1, standard_deviation=2), lowerbound=0, upperbound=10
)
function_forecast = sim.Bounded(
    sim.Normal(mean=20, standard_deviation=20), lowerbound=0, upperbound=120
)
function_lead_time = sim.Bounded(
    sim.Normal(mean=5, standard_deviation=2), lowerbound=0, upperbound=10
)
function_delivery_fullfillment = sim.Bounded(
    sim.Normal(mean=0.75, standard_deviation=0.15), lowerbound=0, upperbound=1
)

# MONITORS


In [4]:
initial_inventory = 100
initial_open_po = 0

monitor_demand = sim.Monitor(name="demand", type="float")
monitor_inventory = sim.Monitor(
    name="inventory", level=True, initial_tally=initial_inventory, type="float"
)
monitor_po = sim.Monitor(name="po", type="float")
monitor_open_po = sim.Monitor(
    name="open_po", level=True, initial_tally=initial_open_po, type="float"
)
monitor_lost_sales = sim.Monitor(name="lost_sales", type="float")

# Animation


In [5]:
sim.AnimateText("Demonstration dynamic AnimateMonitor", fontsize=20, y=700, x=100)

sim.AnimateMonitor(
    monitor_inventory,
    linewidth=3,
    x=100,
    y=100,
    width=900,
    height=250,
    vertical_scale=lambda arg, t: min(50, 250 / arg.monitor().maximum()),
    labels=lambda arg, t: [i for i in range(0, int(arg.monitor().maximum()), 10)],
    horizontal_scale=lambda t: min(10, 900 / t),
)

# sim.AnimateMonitor(
#     monitor_inventory,
#     linewidth=5,
#     x=100,
#     y=600,
#     width=900,
#     height=100,
#     vertical_scale=lambda arg, t: min(50, 250 / arg.monitor().maximum()),
#     labels=lambda arg, t: [i for i in range(0, int(arg.monitor().maximum()), 10)],
#     horizontal_scale=lambda t: min(10, 900 / t),
# )

# sim.AnimateMonitor(
#     monitor_demand,
#     linewidth=5,
#     x=100,
#     y=300,
#     width=900,
#     height=100,
#     vertical_scale=lambda arg, t: min(50, 250 / arg.monitor().maximum()),
#     labels=lambda arg, t: [i for i in range(0, int(arg.monitor().maximum()), 10)],
#     horizontal_scale=lambda t: min((10, 900 / t)),
# )

# CLASSES


In [6]:
class Sales(sim.Component):
    def process(self):
        while True:
            print("start of period inventory: ", monitor_inventory.get())
            demand = function_demand.sample()
            monitor_demand.tally(demand)
            print("daily demand: ", demand)
            monitor_lost_sales.tally(max(0, demand - monitor_inventory.get()))
            monitor_inventory.tally(max(0, monitor_inventory.get() - demand))
            print("end of period inventory: ", monitor_inventory.get())
            self.hold(1)

# FUNCTIONS


In [7]:
def calculate_safety_stock(
    avg_demand, std_demand, avg_lead_time, std_lead_time, service_level
):
    # Calculate the Z-score for the desired service level
    z_score = norm.ppf(service_level)

    # Calculate the safety stock
    safety_stock = z_score * math.sqrt(
        (avg_lead_time * std_demand**2) + (avg_demand**2 * std_lead_time**2)
    )
    return safety_stock

# RUN


In [8]:
# env.speed(10)
Sales()

# env.video_repeat(0)
# env.video("demo dynamic animatemonitor.gif")
# env.run(1)
# env.animate(True)
env.run(200)
# env.video_close()

                                       line numbers prefixed by A refer to  2336438899.py
   A2                                  sales.1 create                       
                                       line numbers prefixed by B refer to  2873576339.py
   A2                                  sales.1 activate                     scheduled for 0.000 @   B2+ process=process
   A8                                  main run +200.000                    scheduled for 200.000 @    8+
   B2+      0.000 sales.1              current                              
start of period inventory:  100
daily demand:  1.3615418191191784
end of period inventory:  98.63845818088082
  B11                                  sales.1 hold +1.000                  scheduled for 1.000 @  B11+
  B11+      1.000 sales.1              current                              
start of period inventory:  98.63845818088082
daily demand:  2.664511610622426
end of period inventory:  95.97394657025839
  B11                     

In [9]:
print("monitor_inventory", monitor_inventory.as_dataframe())
print("monitor_demand", monitor_demand.as_dataframe())
print("monitor_lost_sales", monitor_lost_sales.as_dataframe())

monitor_inventory          t  inventory.x
0      0.0    98.638458
1      1.0    95.973947
2      2.0    93.616288
3      3.0    91.238230
4      4.0    90.666393
..     ...          ...
196  196.0     0.000000
197  197.0     0.000000
198  198.0     0.000000
199  199.0     0.000000
200  200.0     0.000000

[201 rows x 2 columns]
monitor_demand          t  demand.x
0      0.0  1.361542
1      1.0  2.664512
2      2.0  2.357659
3      3.0  2.378058
4      4.0  0.571837
..     ...       ...
196  196.0  3.006000
197  197.0  1.011479
198  198.0  2.373982
199  199.0  0.338736
200  200.0  0.194654

[201 rows x 2 columns]
monitor_lost_sales          t  lost_sales.x
0      0.0      0.000000
1      1.0      0.000000
2      2.0      0.000000
3      3.0      0.000000
4      4.0      0.000000
..     ...           ...
196  196.0      3.006000
197  197.0      1.011479
198  198.0      2.373982
199  199.0      0.338736
200  200.0      0.194654

[201 rows x 2 columns]
